In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
sys.path.append("/kaggle/input/yolov7-lib/yolov7-main")
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import transforms
import numpy as np
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/yolov7-weights'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yolov7-weights/yolov7-e6.pt
/kaggle/input/yolov7-weights/yolov7-d6.pt
/kaggle/input/yolov7-weights/yolov7x.pt
/kaggle/input/yolov7-weights/yolov7-e6e.pt
/kaggle/input/yolov7-weights/yolov7-tiny.pt
/kaggle/input/yolov7-weights/zidane.jpeg
/kaggle/input/yolov7-weights/yolov7-w6-pose.pt
/kaggle/input/yolov7-weights/yolov7.pt
/kaggle/input/yolov7-weights/yolov7-w6.pt
/kaggle/input/yolov7-weights/yolov7-mask.pt


In [2]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# load pre-trained weights
weigths = torch.load('/kaggle/input/yolov7-weights/yolov7-w6-pose.pt', map_location=device)
model = weigths['model']
_ = model.float().eval()
if torch.cuda.is_available():
    model.half().to(device)

In [3]:
import gc
key_points_dict = {}
images_path = '/kaggle/input/dataset/images/'
images = os.listdir(images_path)
#iterate over the images
for img in images:
    #no_grad() to my model tells PyTorch that I don't want to store any previous computations, thus freeing my GPU space.
    with torch.no_grad():
        
        ## Transform image from numpy to torch format    
        image = cv2.imread(images_path+img)
        image = letterbox(image, 640, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        
        if torch.cuda.is_available():
            image = image.half().to(device)   
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        
        keypoints = output_to_keypoint(output)
        # iterate for multiple figures in an image
        for idx in range(keypoints.shape[0]):
            keypoints_list = keypoints[idx,7:].tolist()
        key_points_dict[img] = keypoints_list
        
    del image, output
    torch.cuda.empty_cache()
    gc.collect()
        


/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
key_points_df = pd.DataFrame(key_points_dict.items(),columns=["image_id","key_points"])

In [5]:
key_points_df.head(5)

,image_id,key_points
0,Image_5576.jpg,"[290.25, 195.875, 0.9794921875, 321.25, 199.25..."
1,Image_8010.jpg,"[384.5, 151.5, 0.8916015625, 389.0, 147.25, 0...."
2,Image_5775.jpg,"[290.75, 183.0, 0.9921875, 295.0, 177.625, 0.9..."
3,Image_7411.jpg,"[287.5, 253.75, 0.99560546875, 287.5, 231.875,..."
4,Image_4172.jpg,"[123.0, 229.625, 0.9912109375, 117.375, 214.0,..."


In [6]:
header_list = ["image_id", "labels"]
labels_df = pd.read_csv('/kaggle/input/labels/labels.csv',names=header_list)

In [7]:
labels_df.head(5)

,image_id,labels
0,Image_1.jpg,sitting
1,Image_4.jpg,sleeping
2,Image_6.jpg,sleeping
3,Image_15.jpg,sleeping
4,Image_33.jpg,sitting


In [8]:

unbalanced_keypoints_df = pd.merge(left=key_points_df,right=labels_df,how='inner',on='image_id')

In [9]:
unbalanced_keypoints_df.head(5)

,image_id,key_points,labels
0,Image_5576.jpg,"[290.25, 195.875, 0.9794921875, 321.25, 199.25...",sleeping
1,Image_8010.jpg,"[384.5, 151.5, 0.8916015625, 389.0, 147.25, 0....",running
2,Image_5775.jpg,"[290.75, 183.0, 0.9921875, 295.0, 177.625, 0.9...",sitting
3,Image_7411.jpg,"[287.5, 253.75, 0.99560546875, 287.5, 231.875,...",sleeping
4,Image_4172.jpg,"[123.0, 229.625, 0.9912109375, 117.375, 214.0,...",sleeping


In [10]:
unbalanced_keypoints_df.to_csv('unbalanced_keypoints.csv',index=False)